In [ ]:
import yfinance as yf
import numpy as np
  import pandas as pd
  import tensorflow as tf
  import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
# Data preprocessing
data= yf.download('BTC-USD', period="max", interval="1d")
data = data.dropna()
data = data.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'])
data = data.rename(columns={'Close': 'Price'})

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data['Price'] = scaler.fit_transform(data[['Price']])

# Add MACD calculation to the DataFrame
macd, macd_signal, macd_hist = ta.MACD(data['Price'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['MACD'] = scaler.fit_transform(macd.reshape(-1, 1))
data['MACD_Signal'] = scaler.fit_transform(macd_signal.reshape(-1, 1))
data['MACD_Hist'] = scaler.fit_transform(macd_hist.reshape(-1, 1))

# Add buy/sell signal based on MACD divergence/convergence
data['Signal'] = np.where(data['MACD'] > data['MACD_Signal'], 1, 0)
data['Signal'] = np.where(data['MACD'] < data['MACD_Signal'], -1, data['Signal'])

# Define the number of steps to look ahead
steps_ahead = 30

# Split data into training and testing sets
x = np.array(data.drop(columns=['Signal']).values)
y = np.array(data['Signal'].values)
x = np.reshape(x, (x.shape[0], x.shape[1], 1))

# Initialize the list to store accuracy values
accuracies = []
f1_scores = []

# Start the walk-forward validation loop
for i in range(0, len(x) - steps_ahead, steps_ahead):
    x_train = x[:i + steps_ahead]
    y_train = y[:i + steps_ahead]
    x_test = x[i + steps_ahead:i + 2 * steps_ahead]
    y_test = y[i + steps_ahead:i + 2 * steps_ahead]
    
    # Create the neural network
    model = tf.keras.Sequential([
      tf.keras.layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True),
      tf.keras.layers.LSTM(64, return_sequences=True),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.LSTM(32),
      tf.keras.layers.Dense(16, activation='relu'),
      tf.keras.layers.Dense(8, activation='relu'),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])
    # Compile the neural network
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the neural network
    model.fit(x_train, y_train, epochs=10, batch_size=32)

    # Make predictions with the neural network
    predictions = model.predict(x_test)
    predictions = np.where(predictions > 0.5, 1, -1)

    # Evaluate the neural network on the test data
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print('Test Loss: ', test_loss)
    print('Test Accuracy: ', test_accuracy)
    
    # Calculate the F1 score
    f1 = f1_score(y_test, predictions, average='macro')
    print('F1 Score: ', f1)
    
    # Add the F1 score to the list
    f1_scores.append(f1)
    # Calculate the average F1 score
    avg_f1_score = sum(f1_scores) / len(f1_scores)
    print('Average F1 Score: ', avg_f1_score)
    # Add the accuracy to the list
    accuracies.append(test_accuracy)

# Calculate the average accuracy
avg_accuracy = sum(accuracies) / len(accuracies)
print('Average Accuracy: ', avg_accuracy)